In [ ]:
# !pip install langchain
# !pip install pinecone-client
# !pip install pypdf
# !pip install openai
# !pip install tiktoken

In [27]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv
import os
import sys

In [3]:
!mkdir pdfs

In [5]:
loader = PyPDFDirectoryLoader('pdfs')

In [6]:
data = loader.load()

In [17]:
print(data[0])

page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object
detectors
Chien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1
1Institute of Information Science, Academia Sinica, Taiwan
kinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw
Abstract
YOLOv7 surpasses all known object detectors in both
speed and accuracy in the range from 5 FPS to 160 FPS
and has the highest accuracy 56.8% AP among all known
real-time object detectors with 30 FPS or higher on GPU
V100. YOLOv7-E6 object detector (56 FPS V100, 55.9%
AP) outperforms both transformer-based detector SWIN-
L Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by
509% in speed and 2% in accuracy, and convolutional-
based detector ConvNeXt-XL Cascade-Mask R-CNN (8.6
FPS A100, 55.2% AP) by 551% in speed and 0.7% AP
in accuracy, as well as YOLOv7 outperforms: YOLOR,
YOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable
DETR, DINO-5scale-R50, ViT-Adapter-B and many other
object detectors

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=20)

In [8]:
text_chunks = text_splitter.split_documents(data)

In [21]:
text_chunks

[Document(metadata={'source': 'pdfs\\yolov7.pdf', 'page': 0}, page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known'),
 Document(metadata={'source': 'pdfs\\yolov7.pdf', 'page': 0}, page_content='real-time object detectors with 30 FPS or higher on GPU\nV100. YOLOv7-E6 object detector (56 FPS V100, 55.9%\nAP) outperforms both transformer-based detector SWIN-\nL Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by\n509% in speed and 2% in accuracy, and convolutional-\nbased detector ConvNeXt-XL Cascade-Mask R-CNN (8.6\nFPS A100, 55.2% AP) by 551% in speed and 0.7% AP\nin accurac

In [24]:
print(text_chunks[0].page_content)

YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object
detectors
Chien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1
1Institute of Information Science, Academia Sinica, Taiwan
kinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw
Abstract
YOLOv7 surpasses all known object detectors in both
speed and accuracy in the range from 5 FPS to 160 FPS
and has the highest accuracy 56.8% AP among all known


In [25]:
print(text_chunks[1].page_content)

real-time object detectors with 30 FPS or higher on GPU
V100. YOLOv7-E6 object detector (56 FPS V100, 55.9%
AP) outperforms both transformer-based detector SWIN-
L Cascade-Mask R-CNN (9.2 FPS A100, 53.9% AP) by
509% in speed and 2% in accuracy, and convolutional-
based detector ConvNeXt-XL Cascade-Mask R-CNN (8.6
FPS A100, 55.2% AP) by 551% in speed and 0.7% AP
in accuracy, as well as YOLOv7 outperforms: YOLOR,
YOLOX, Scaled-YOLOv4, YOLOv5, DETR, Deformable


In [26]:
len(text_chunks)

161

In [29]:
print(text_chunks[151].page_content)

A. Appendix
A.1. Implementation details
A.1.1 Architectures
Figure A1 and Figure A2 show the architectures of the pro-
posed YOLOv7 P5 and YOLOv7 P6, respectively. Please
note that Down, Up, CSPSPP, and the compression rate
of a transition layer of ELAN modules in different models
have a little bit different implementation. The details can be
found at https://github.com/WongKinYiu/yolov7/tree/main/
cfg/deploy.
Figure A1: Architectures of YOLOv7 P5 models.


In [9]:
load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY_')

In [10]:
os.environ["OPENAI_API_KEY"] = openai_key

In [11]:
embedding = OpenAIEmbeddings()

In [54]:
embedding.embed_query("How are you")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-Tw9KZ***************************************2Ezi. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [12]:
pinecone_key = os.getenv('PINECONE_API_KEY')

In [14]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', pinecone_key)

In [22]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [23]:
index_name = 'testing'

index = pc.Index(index_name)

## Create Embeddings for each of the Text Chunk

In [25]:
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], index_name=index_name, embedding=embedding)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-Tw9KZ***************************************2Ezi. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [26]:
docsearch

NameError: name 'docsearch' is not defined

In [ ]:
docsearch.as_retriever()

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
docs = docsearch.similarity_search(query)

In [ ]:
llm = OpenAI()

## With the retrievalQA, we can run query through our vector db embeddings

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
qa.run(query)

In [ ]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")